In [22]:
from pathlib import Path

# Resolver raíz del repo sin usar parents indexados
# Estrategia: partir de cwd y subir hasta encontrar carpeta "data" y "notebooks".
# Si no se encuentran, usar cwd como raíz.
def resolve_repo_root(start: Path) -> Path:
    current = start
    for _ in range(6):  # subir hasta 6 niveles por seguridad
        if (current / 'data').exists() and (current / 'notebooks').exists():
            return current
        if current.parent == current:
            break
        current = current.parent
    return start

root = resolve_repo_root(Path.cwd())
data_dir = root / 'data' / 'raw'
data_dir.mkdir(parents=True, exist_ok=True)


## 🔧 Instalación rápida de librerías (ejecuta en Terminal VS Code)

```pwsh
# Base análisis y mapas
F:\GitHub\supply-chain-data-notebooks\.venv\Scripts\python.exe -m pip install pandas numpy geopandas folium leafmap plotly pytz requests

# Lectura GTFS
F:\GitHub\supply-chain-data-notebooks\.venv\Scripts\python.exe -m pip install partridge gtfs_kit

# Opcional: validaciones y protobuf
F:\GitHub\supply-chain-data-notebooks\.venv\Scripts\python.exe -m pip install protobuf fastparquet pyarrow
```

Tras instalar, ejecuta las celdas en orden para ver resultados visibles (lectura GTFS y mapa HTML).

# 🚌 TR-01: Transporte masivo (LATAM) – GTFS/GTFS-RT y métricas operativas

Este cuaderno guía la ingestión y análisis de datos GTFS/GTFS-RT aplicados al contexto LATAM, con foco en validación de esquema, procesamiento geoespacial, métricas de operación y visualizaciones.

## 1) Configurar entorno y dependencias

- Librerías: `pandas`, `geopandas`, `numpy`, `matplotlib`, `folium`/`leafmap`, `partridge` o `gtfs_kit`, `pytz`, `plotly`.
- Instalar en el venv del repo (VS Code Terminal):
```pwsh
F:\GitHub\supply-chain-data-notebooks\.venv\Scripts\python.exe -m pip install geopandas folium leafmap partridge pytz plotly
```
- Cargar librerías en las celdas de código siguientes.

In [23]:
# Placeholder: importación de librerías (ajuste según tu entorno)
import pandas as pd
import numpy as np

## 2) Cargar y validar paquete GTFS (esquema)
Archivos esperados: `agency.txt`, `routes.txt`, `trips.txt`, `stops.txt`, `stop_times.txt`, `shapes.txt`, `calendar.txt` y/o `calendar_dates.txt`. Validar presencia y tipos de columnas clave (`route_id`, `trip_id`, `stop_id`).

In [24]:
# Placeholder: lectura GTFS (sin implementar)
# Sugerencia: usar partridge o gtfs_kit para leer el ZIP y acceder a tablas.
gtfs_zip_path = "path/a/tu_feed_gtfs.zip"
# TODO: implementar lectura con partridge/gtfs_kit y validaciones de columnas obligatorias.

## 3) Normalizar y limpiar datos para LATAM
- Normalizar encoding UTF-8 y nombres de `agency`, resolver zonas horarias (`America/Santiago`, `America/Mexico_City`, `America/Bogota`).
- Mapear `route_type` locales a estándar GTFS.
- Unificar formatos de `stop_id`/`trip_id`.
- Integrar feriados locales desde `calendar_dates.txt`.

## 4) Construir tablas relacionales: agency, routes, trips, stops, shapes
- Unir `trips` con `routes` y `shapes`.
- Generar índices y columnas para análisis (ej. `servicio_diario`, `sentido`, `vehiculo_tipo`).
- Snippets para joins eficientes con `pandas`/`geopandas`.

## 5) Generar geometrías y GeoDataFrames (paradas y trazas)
- Crear `GeoDataFrame` de paradas (Point) y trazas de ruta (LineString) a partir de `shapes.txt`.
- Proyectar a CRS métricos (ej. EPSG:3857 / CRS locales).
- Simplificación de trazas y cálculo de distancias entre paradas.

## 6) Calcular métricas operativas básicas (headway, span, trips/día)
- Headway por línea/segmento: media de diferencias entre salidas, $h=\mathrm{mean}(\Delta t)$.
- Span de servicio: primer/último viaje.
- Trips/día por `route_id` y cobertura espacial.

## 7) Puntualidad y desempeño usando GTFS-RT o simulación
- Ingesta básica de GTFS-RT (posiciones) o simulación de retrasos.
- Vincular con `trips` GTFS y calcular puntualidad.
- OTP: $$\mathrm{OTP}=\frac{N_{\text{on-time}}}{N_{\text{total}}}$$, umbrales típicos ±5 min; métricas por hora/segmento.

In [25]:
# Placeholder: GTFS-RT (sin implementar)
gtfs_rt_endpoint = "url/a/gtfs-rt.pb"
# TODO: fetch y parse de protobuf/JSON, vínculo con trips y cálculo de puntualidad.

## 8) Estimación de capacidad y niveles de servicio
- Capacidad por viaje usando tablas de vehículo o supuestos.
- Cálculo de **load factor** y nivel de servicio por parada y por hora.
- Métricas: pasajeros estimados / capacidad, frecuencia efectiva.

## 9) Visualizaciones interactivas y paneles
- Mapas con `folium`/`leafmap` para rutas/paradas.
- Series temporales (headways, OTP) con `plotly`.
- Heatmaps de cobertura y demanda estimada.
- Exportar HTML interactivo.

## 10) Exportar métricas y reportes
- Guardar tablas resumen en CSV/JSON y GeoPackage.
- Comandos para escribir en disco y visualizar resultados en VS Code Output.

## 11) Pruebas unitarias y verificación en VS Code
- Tests con `pytest` para funciones críticas: validación de esquema, cálculo de headway, vínculo GTFS-RT.
- Ejecutar en el runner integrado y revisar el panel de pruebas.

## 📎 Feeds GTFS públicos (referencia LATAM)
- Chile (Santiago): https://www.dtpm.cl/index.php/gtfs-transantiago/ 
- México (CDMX): https://datos.cdmx.gob.mx/explore/?sort=modified&tags=gtfs 
- Brasil (São Paulo/EMTU/Metrô): https://www.sptrans.com.br/desenvolvedores/ e https://www.metro.sp.gov.br/data/gtfs/ 
- Colombia (Bogotá): https://datosabiertos.bogota.gov.co/ (buscar GTFS) 

Puedes usar cualquiera de estos para pruebas locales.

In [27]:
# 🔽 Descarga de un GTFS de muestra (local)
# Requisitos: requests instalado. Si falla la URL, usa un archivo GTFS local.
import os
from pathlib import Path
import requests
import zipfile

# Usar la raíz del repo resuelta previamente
# variable 'root' y 'data_dir' vienen de la celda inicial

# Reemplaza por una URL real del listado de feeds LATAM
sample_url = os.environ.get('GTFS_SAMPLE_URL', 'https://gtfs.zip.example/your_feed.zip')
out_zip = data_dir / 'gtfs_sample.zip'
local_fallback = data_dir / 'gtfs_local.zip'  # si ya tienes uno local, colócalo aquí

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) GitHubCopilot/Notebook',
    'Accept': 'application/zip,application/octet-stream,*/*'
}

def is_valid_zip(path: Path) -> bool:
    try:
        with zipfile.ZipFile(path) as zf:
            names = set(zf.namelist())
            required_any = {'stops.txt','routes.txt','trips.txt'}
            return any(name in names for name in required_any)
    except Exception:
        return False

download_attempted = False
if 'gtfs.zip.example' not in sample_url:
    try:
        download_attempted = True
        with requests.get(sample_url, headers=headers, timeout=90, allow_redirects=True, stream=True) as r:
            r.raise_for_status()
            with open(out_zip, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
        if is_valid_zip(out_zip):
            print(f"Descargado y validado: {out_zip}")
        else:
            print("El ZIP descargado no parece contener tablas GTFS válidas. Intentando fallback local...")
            out_zip.unlink(missing_ok=True)
    except Exception as e:
        print("Descarga fallida:", e)

if not out_zip.exists():
    if local_fallback.exists() and is_valid_zip(local_fallback):
        out_zip.write_bytes(local_fallback.read_bytes())
        print(f"Usando fallback local: {local_fallback} -> {out_zip}")
    else:
        if not download_attempted:
            print("Define una URL válida en sample_url o setea la variable de entorno GTFS_SAMPLE_URL.")
        print("Coloca un ZIP GTFS en:", local_fallback)

if out_zip.exists() and is_valid_zip(out_zip):
    print("GTFS listo:", out_zip)
else:
    print("No hay GTFS descargado/validado. Revisa la URL o agrega un ZIP local.")

GTFS listo: f:\GitHub\supply-chain-data-notebooks\data\raw\gtfs_sample.zip


## 📄 Lectura GTFS con partridge/gtfs_kit (mínimo)
Intentamos cargar el ZIP local `data/raw/gtfs_sample.zip` leyendo tablas clave (`stops`, `routes`, `trips`). Si no están instalados, se muestra un mensaje con instrucciones.

In [28]:
# Intento de lectura con partridge; si falla, probar gtfs_kit
from pathlib import Path
zip_path = (root / 'data' / 'raw' / 'gtfs_sample.zip')

stops = routes = trips = None

try:
    import partridge as pt
    feed = pt.load_feed(zip_path)
    stops = feed.stops
    routes = feed.routes
    trips = feed.trips
    print("Leído con partridge:", len(stops), "paradas;", len(routes), "rutas;", len(trips), "trips")
except Exception as e:
    print("partridge no disponible o fallo:", e)
    try:
        import gtfs_kit as gk
        import zipfile
        with zipfile.ZipFile(zip_path) as zf:
            stops = gk.read_table(zf, 'stops')
            routes = gk.read_table(zf, 'routes')
            trips = gk.read_table(zf, 'trips')
        print("Leído con gtfs_kit:", len(stops), "paradas;", len(routes), "rutas;", len(trips), "trips")
    except Exception as e2:
        print("gtfs_kit no disponible o fallo:", e2)
        print("Instala dependencias:")
        print("F:/GitHub/supply-chain-data-notebooks/.venv/Scripts/python.exe -m pip install partridge gtfs_kit")

Leído con partridge: 2 paradas; 1 rutas; 3 trips


## 🗺️ Mapa rápido de paradas (folium)
Si se cargaron `stops`, creamos un mapa sencillo centrado en la primera parada y lo guardamos como HTML.

In [29]:
# Generar mapa si hay stops disponibles
try:
    import folium
    from pathlib import Path
    if stops is not None and len(stops) > 0:
        lat_col = 'stop_lat' if 'stop_lat' in stops.columns else None
        lon_col = 'stop_lon' if 'stop_lon' in stops.columns else None
        if lat_col and lon_col:
            center = [float(stops.iloc[0][lat_col]), float(stops.iloc[0][lon_col])]
            m = folium.Map(location=center, zoom_start=12)
            for _, row in stops.head(200).iterrows():
                try:
                    folium.CircleMarker(
                        location=[float(row[lat_col]), float(row[lon_col])],
                        radius=3,
                        color='blue',
                        fill=True,
                        fill_opacity=0.7,
                        popup=str(row.get('stop_name', 'stop'))
                    ).add_to(m)
                except Exception:
                    pass
            out_html = (root / 'data' / 'processed' / 'gtfs_stops_map.html')
            out_html.parent.mkdir(parents=True, exist_ok=True)
            m.save(str(out_html))
            print("Mapa guardado:", out_html)
        else:
            print("No se encontraron columnas stop_lat/stop_lon en stops.")
    else:
        print("No hay stops cargados para mapear.")
except Exception as e:
    print("No se pudo generar el mapa:", e)

Mapa guardado: f:\GitHub\supply-chain-data-notebooks\data\processed\gtfs_stops_map.html


## 📈 Ejemplo ejecutable: headways simples con `stop_times`
Si el feed incluye `stop_times.txt`, podemos estimar headways por `route_id` usando diferencias de tiempo entre salidas (demo simplificada).

In [30]:
# Demo headways (si hay stop_times)
try:
    import zipfile, pandas as pd
    zip_path = root / 'data' / 'raw' / 'gtfs_sample.zip'
    with zipfile.ZipFile(zip_path) as zf:
        if 'stop_times.txt' in zf.namelist() and trips is not None:
            st = pd.read_csv(zf.open('stop_times.txt'))
            # convertir hora a segundos
            def h2s(h):
                try:
                    hh, mm, ss = map(int, str(h).split(':'))
                    return hh*3600 + mm*60 + ss
                except Exception:
                    return None
            st['dep_s'] = st['departure_time'].apply(h2s)
            df = st.merge(trips[['trip_id','route_id']], on='trip_id', how='left')
            # headway por route_id: diferencias promedio entre dep_s por trip
            headways = (
                df.dropna(subset=['dep_s'])
                  .sort_values(['route_id','dep_s'])
                  .groupby('route_id')['dep_s']
                  .apply(lambda x: pd.Series(x).diff().dropna().mean()/60.0)
            )
            hw = headways.reset_index(name='headway_min')
            print(hw.head(10))
        else:
            print('No hay stop_times.txt o trips cargados para la demo de headways.')
except Exception as e:
    print('Fallo headways demo:', e)

  route_id  headway_min
0       R1          6.0
